## [Computational Social Science] Project 5: Natural Language Processing

Student: Brenda Sciepura

In this project, you will use natural language processing techniques to explore a dataset containing tweets from members of the 116th United States Congress that met from January 3, 2019 to January 2, 2021. The dataset has also been cleaned to contain information about each legislator. Concretely, you will do the following:

* Preprocess the text of legislators' tweets
* Conduct Exploratory Data Analysis of the text
* Use sentiment analysis to explore differences between legislators' tweets
* Featurize text with manual feature engineering, frequency-based, and vector-based techniques
* Predict legislators' political parties and whether they are a Senator or Representative

You will explore two questions that relate to two central findings in political science and examine how they relate to the text of legislators' tweets. First, political scientists have argued that U.S. politics is currently highly polarized relative to other periods in American history, but also that the polarization is asymmetric. Historically, there were several conservative Democrats (i.e. "blue dog Democrats") and liberal Republicans (i.e. "Rockefeller Republicans"), as measured by popular measurement tools like [DW-NOMINATE](https://en.wikipedia.org/wiki/NOMINATE_(scaling_method)#:~:text=DW\%2DNOMINATE\%20scores\%20have\%20been,in\%20the\%20liberal\%2Dconservative\%20scale.). However, in the last few years, there are few if any examples of any Democrat in Congress being further to the right than any Republican and vice versa. At the same time, scholars have argued that this polarization is mostly a function of the Republican party moving further right than the Democratic party has moved left. **Does this sort of asymmetric polarization show up in how politicians communicate to their constituents through tweets?**

Second, the U.S. Congress is a bicameral legislature, and there has long been debate about partisanship in the Senate versus the House. The House of Representatives is apportioned by population and all members serve two year terms. In the Senate, each state receives two Senators and each Senator serves a term of six years. For a variety of reasons (smaller chamber size, more insulation from the voters, rules and norms like the filibuster, etc.), the Senate has been argued to be the "cooling saucer" of Congress in that it is more bipartisan and moderate than the House. **Does the theory that the Senate is more moderate have support in Senators' tweets?**

**Note**: See the project handout for more details on caveats and the data dictionary.

In [ ]:
# download libraries
# ----------
#!pip install spacy
#!python -m spacy download en_core_web_sm
#!pip install scattertext
#!pip install wordcloud
#!pip install textblob
#!pip install adjustText

In [ ]:
# pandas and numpy
import pandas as pd
import numpy as np

# punctuation, stop words and English language model
from string import punctuation
from spacy.lang.en.stop_words import STOP_WORDS
import en_core_web_sm
nlp = en_core_web_sm.load()

# countvectorizer, tfidfvectorizer
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# gensim
import gensim
from gensim import models

# textblob
from textblob import TextBlob

## LDA
from sklearn.decomposition import NMF, LatentDirichletAllocation

# load wordcloud
from wordcloud import WordCloud, ImageColorGenerator

# plotting
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

# for kMeans and silhouette scores
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
import matplotlib.cm as cm

# for the classification task 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import LabelBinarizer

# load libraires
import random
from adjustText import adjust_text
from sklearn.manifold import TSNE

# load STOP_WORDS module from spaCy library
# ----------
from spacy.lang.en.stop_words import STOP_WORDS

# load punctuation module from string library
# ----------
from string import punctuation  # note that this is a base library

In [ ]:
# load data 
# ----------
congress_tweets = pd.read_csv("data/116th Congressional Tweets and Demographics.csv")
congress_tweets.head()

## Preprocessing

The first step in working with text data is to preprocess it. Make sure you do the following:

* Remove punctuation and stop words. The `rem_punc_stop()` function we used in lab is provided to you but you should feel free to edit it as necessary for other steps
* Remove tokens that occur frequently in tweets, but may not be helpful for downstream classification. For instance, many tweets contain a flag for retweeting, or share a URL 

As you search online, you might run into solutions that rely on regular expressions. You are free to use these, but you should also be able to preprocess using the techniques we covered in lab. Specifically, we encourage you to use spaCy's token attributes and string methods to do some of this text preprocessing.

In [ ]:
## defining function to remove punctuation, stop words and other pre-defined characters 
def rem_punc_stop(text):
    
    ## tokenize
    doc = nlp(text)
    
    ## extract tokens (not including tokens if they are stop words, spaces or punctuation, and I'm also removing any alpha numeric characters)
    spacy_words = [token.text for token in doc if not token.is_stop and not token.is_space and not token.is_punct and (token.is_alpha or token.is_digit)]
    
    ## removing URL 
    spacy_words = [word for word in spacy_words if not word.startswith('http')]
    
    # remove flags for retweeting (e.g., RT)
    spacy_words = [word for word in spacy_words if word.lower() != 'rt']
    
    # remove another characters that appear very frequently in tweets (QT)
    spacy_words = [word for word in spacy_words if word.lower() != 'qt']
    
    # remove "amp" which comes after each & in the text (&amp)
    spacy_words = [word for word in spacy_words if word.lower() != 'amp']
     
    return spacy_words    

In [ ]:
## here I was checking why "amp" appeared so many times and then I decided to remove it 

## tweets_with_amp = subset_df[subset_df['text'].str.contains("amp")]
## tweets_with_amp_text = tweets_with_amp['text']

# Print only the first 20 tweets
## for tweet_text in tweets_with_amp_text.head(10): print(tweet_text)

In [ ]:
# subset the df to 7,000 random rows
subset_df = congress_tweets.sample(n=7000)

# apply the function to all columns 
subset_df['tokens'] = subset_df['text'].map(lambda x: rem_punc_stop(x)) 
subset_df['tokens'] 

In [ ]:
## removing independents from dataset to have a binary outcome
subset_df = subset_df[subset_df['party'] != 'Independent']

In [ ]:
## I'm creating this here so that I can use the tokenized version or the full text version as needed 
subset_df['tokens_full_text'] = subset_df['tokens'].map(lambda text: ' '.join(text))

## Exploratory Data Analysis

Use two of the techniques we covered in lab (or other techniques outside of lab!) to explore the text of the tweets. You should construct these visualizations with an eye toward the eventual classification tasks: (1) predicting the legislator's political party based on the text of their tweet, and (2) predicting whether the legislator is a Senator or Representative. As a reminder, in lab we covered word frequencies, word clouds, word/character counts, scattertext, and topic modeling as possible exploration tools. 

### EDA 1: WordClouds

In [ ]:
# apply function to text object
text = ' '.join(subset_df['tokens'].map(lambda text: ' '.join(text)))

# create WordCloud visualization using the "text" object by 
wordcloud = WordCloud(background_color = "black",  
                      colormap='rainbow',
                      random_state=3141).generate(text)              

# plot 
plt.imshow(wordcloud, interpolation = 'bilinear') 
plt.axis('off')                        
plt.show()                             

In [ ]:
## I create two distinct wordclouds: one representing tweets coming from republicans and the other tweets coming from democrats

# Filter data for Republicans
republican_text = ' '.join(subset_df[subset_df['party'] == 'Republican']['tokens'].map(lambda text: ' '.join(text)))

# Create WordCloud for Republicans
republican_wordcloud = WordCloud(background_color="black", 
                                 colormap='rainbow', 
                                 random_state=3141).generate(republican_text)

# Plot Republican WordCloud
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(republican_wordcloud, interpolation='bilinear')
plt.title('Republicans')
plt.axis('off')

# Filter data for Democrats
democrat_text = ' '.join(subset_df[subset_df['party'] == 'Democrat']['tokens'].map(lambda text: ' '.join(text)))

# Create WordCloud for Democrats
democrat_wordcloud = WordCloud(background_color="grey", 
                               colormap='rainbow', 
                               random_state=3141).generate(democrat_text)

# Plot Democrat WordCloud
plt.subplot(1, 2, 2)
plt.imshow(democrat_wordcloud, interpolation='bilinear')
plt.title('Democrats')
plt.axis('off')

# Show the plot
plt.tight_layout()
plt.show()

## Comments on wordclouds: 
## It's interesting that "thank" is way more used by republicans. And that democrat stands out in republicans' tweets whereas Trump stands out in democrat tweets. And the word "act" stands out more in democtats' tweets.

In [ ]:
## I create two distinct wordclouds: one representing tweets coming from Senators and the other tweets coming from Representatives

# Filter data for Representatives
representative_text = ' '.join(subset_df[subset_df['position'] == 'Rep']['tokens'].map(lambda text: ' '.join(text)))

# Create WordCloud for Representatives
representative_wordcloud = WordCloud(background_color="black", 
                                 colormap='rainbow', 
                                 random_state=3141).generate(representative_text)

# Plot Representatives WordCloud
plt.figure(figsize=(10, 5))
plt.subplot(1, 2, 1)
plt.imshow(representative_wordcloud, interpolation='bilinear')
plt.title('Representatives')
plt.axis('off')

# Filter data for Senators
senator_text = ' '.join(subset_df[subset_df['position'] == 'Sen']['tokens'].map(lambda text: ' '.join(text)))

# Create WordCloud for Senators
senator_wordcloud = WordCloud(background_color="grey", 
                               colormap='rainbow', 
                               random_state=3141).generate(senator_text)

# Plot Senators WordCloud
plt.subplot(1, 2, 2)
plt.imshow(senator_wordcloud, interpolation='bilinear')
plt.title('Senators')
plt.axis('off')

# Show the plot
plt.tight_layout()
plt.show()

## Comments on wordclouds: 
## Representatives' graph is closer to Republicans' graph, whereas Senatos' graph is closer to Democrats' graph. 

## EDA 2: Word/Character Count

In [ ]:
# count number of characters -- given that these are tweets and there are rules about lenght maybe this is not so important
subset_df['tweet_length'] = subset_df['text'].apply(len)

# create histogram of tweet length (number of characters)
sns.displot(subset_df, x="tweet_length")
plt.show()

In [ ]:
# count number of words
subset_df['word_count'] = subset_df['text'].apply(lambda x: len(str(x).split()))

# create histogram of tweet length (number of characters)
sns.displot(subset_df, x="word_count")
plt.show()

### EDA 3: Topic modeling

In [ ]:
# create tf-idf matrix
# ----------

# set X dataset
X = subset_df['text']        

# initialize tf-idf using our preprocessing function
tf = TfidfVectorizer(tokenizer = rem_punc_stop, # use our function for tokenizing created above
                     token_pattern = None)      # set to "None" since we have specify our own pattern

# fit and transform data
tfidf_matrix = tf.fit_transform(X)

In [ ]:
# convert matrix to an arrray and then to a dataframe
tfidf_df = pd.DataFrame(data = tfidf_matrix.toarray(),       # convert to array than to datafram
                         columns=tf.get_feature_names_out()) # specify column names as feature names from TF vectorizer

# sort by term frequency on the first document
tfidf_df.T.nlargest(10,  # transpose the matrix = columns become documents and rows are words
                     0)  # on column index 0 to show the largest

In [ ]:
# K-means clustering
# ------------------------------

# implement kmeans clustering
# ----------
kmeans = KMeans(n_clusters=3,       # specify # of clusters
                max_iter=300        # specify # of iterations
                ).fit(tfidf_matrix) # specify data to fit

# append labels to dataframe
# ----------
subset_df['cluster']= kmeans.labels_    # add labels to original data frame
subset_df.head()

In [ ]:
# determining optional k: elbow method
# ------------------------------

# settings
# ---------
# set an empty list 
Sum_of_squared_distances = []

# set range of k
K = range(1, 10) # second number is the ceiling of the range, but remember it is exclusive


# loop over k
# ---------
for k in K:
    km = KMeans(n_clusters=k,     # number of clusters
                init='k-means++', # method for initalization 
                n_init=10)        # number of times the k-means algorithm is run with different centroid seeds
    km = km.fit(tfidf_matrix)     # fit
    Sum_of_squared_distances.append(km.inertia_) # pipe inertia calculations into list


# plot results
# ---------
plt.plot(K, Sum_of_squared_distances, 'bx-')
plt.xlabel('k')
plt.xticks(range(1, max(K) + 1, 1))
plt.ylabel('Sum_of_squared_distances')
plt.title('Elbow Method For Optimal k')
plt.show()

## the less steep drop happens at 6

In [ ]:
# determining optional k: silhouette scores
# ------------------------------

# iterate over a k-means fits to have different clusters
# ---------
def run_KMeans(max_k, data):
    max_k += 1
    kmeans_results = dict()
    for k in range(2 , max_k):
        kmeans = KMeans(n_clusters = k
                               , init = 'k-means++'
                               , n_init = 10
                               , random_state = 1)

        kmeans_results.update( {k : kmeans.fit(data)} )
        
    return kmeans_results


# calculate average silhouettes scores 
# ---------
# plot silhouettes scores
def printAvg(avg_dict):
    for avg in sorted(avg_dict.keys(), reverse=True):
        print("Avg: {}\tK:{}".format(avg.round(4), avg_dict[avg]))

# plot silhouettes scores       
def plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg): 
    fig, ax1 = plt.subplots(1)
    fig.set_size_inches(8, 6)
    ax1.set_xlim([-0.2, 1])   # play with this to set x-axis limits
    ax1.set_ylim([0, len(df) + (n_clusters + 1) * 10])
    
    ax1.axvline(x=silhouette_avg, color="red", linestyle="--") # The vertical line for average silhouette score of all the values
    ax1.set_yticks([])  # Clear the yaxis labels / ticks
    ax1.set_xticks([-0.2, 0, 0.2, 0.4, 0.6, 0.8, 1])
    plt.title(("Silhouette analysis for K = %d" % n_clusters), fontsize=10, fontweight='bold')
    
    y_lower = 10
    sample_silhouette_values = silhouette_samples(df, kmeans_labels) # Compute the silhouette scores for each sample
    for i in range(n_clusters):
        ith_cluster_silhouette_values = sample_silhouette_values[kmeans_labels == i]
        ith_cluster_silhouette_values.sort()

        size_cluster_i = ith_cluster_silhouette_values.shape[0]
        y_upper = y_lower + size_cluster_i

        color = cm.nipy_spectral(float(i) / n_clusters)
        ax1.fill_betweenx(np.arange(y_lower, y_upper), 0, ith_cluster_silhouette_values, facecolor=color, edgecolor=color, alpha=0.7)

        ax1.text(-0.05, y_lower + 0.5 * size_cluster_i, str(i)) # Label the silhouette plots with their cluster numbers at the middle
        y_lower = y_upper + 10  # Compute the new y_lower for next plot. 10 for the 0 samples
    plt.show()
    
# put it altogether
def silhouette(kmeans_dict, df, plot=True):
    df = df.to_numpy()
    avg_dict = dict()
    for n_clusters, kmeans in kmeans_dict.items():      
        kmeans_labels = kmeans.predict(df)
        silhouette_avg = silhouette_score(df, kmeans_labels) # Average Score for all Samples
        avg_dict.update( {silhouette_avg : n_clusters} )
    
        if(plot): plotSilhouette(df, n_clusters, kmeans_labels, silhouette_avg)

In [ ]:
# calculate silhouette scores across potential k-means clusters 
# ---------

# set K
k = 6 # choose 7 based on the elbow method result from above

# run the k-means algorithm
kmeans_results = run_KMeans(k,                # set k
                            data = tfidf_df)  # identify data


# plot the silhouette analysis
silhouette(kmeans_results,     # take k-means results
           tfidf_df)           # use the dataframe version

In [ ]:
# cluster analysis of top words in each cluster
# ------------------------------

# get the top features from each cluster
def get_top_features_cluster(tf_idf_array, prediction, n_feats):
    labels = np.unique(prediction)
    dfs = []
    for label in labels:
        id_temp = np.where(prediction==label) # indices for each cluster
        x_means = np.mean(tf_idf_array[id_temp], axis = 0) # returns average score across cluster
        sorted_means = np.argsort(x_means)[::-1][:n_feats] # indices with top 20 scores
        features = tf.get_feature_names_out()
        best_features = [(features[i], x_means[i]) for i in sorted_means]
        df = pd.DataFrame(best_features, columns = ['features', 'score'])
        dfs.append(df)
    return dfs

# plot them on a barplot
def plotWords(dfs, n_feats):
    plt.figure(figsize=(8, 4))
    for i in range(0, len(dfs)):
        plt.title(("Most Common Words in Cluster {}".format(i)), fontsize=10, fontweight='bold')
        sns.barplot(x = 'score' , y = 'features', orient = 'h' , data = dfs[i][:n_feats])
        plt.show()

In [ ]:
# run k-means and visualize word count
# ---------

# get results from 5 clusters 
best_result = 5
kmeans = kmeans_results.get(best_result)

# processing for plot
tfidf_array = tfidf_df.to_numpy()     # convert dataframe to array
prediction = kmeans.predict(tfidf_df) # predict cluster using tf-idf dataframe


# plot
n_feats = 20
dfs = get_top_features_cluster(tfidf_array, # specify dataset which is an array
                               prediction,  # make specify prediciton
                               n_feats )    # set number of features 
plotWords(dfs, # specify data for plotting  
          13)  # set number of features

In [ ]:
# define a function to print the top words that we'll use in our model
# --------

def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #{}:".format(topic_idx))
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [ ]:
# Latent Dirichlet Allocation (LDA)
# ---------------------------------------

# pre-processing
# --------
# create a new data object called X
X = subset_df['tokens_full_text']

# initialize tf-idf function and set parameters
tf = TfidfVectorizer(tokenizer = rem_punc_stop,  # specify our function for remove punc and stop words
                     token_pattern = None)       # specify "None" to remove warning. Is this necessary?

In [ ]:
# apply tf-idf vectorizer to our data (X)
tfidf_matrix = tf.fit_transform(X)

In [ ]:
# modify the output to be a dense matrix
dense_matrix = tfidf_matrix.todense()

In [ ]:
# intitialize LDA model and 
# --------
# initialize LDA and set model parameters
lda = LatentDirichletAllocation(n_components=5, # specify the number of components
                                max_iter=20,    # specify the number of iterations 
                                random_state=0) # set a seed for reproducibility

# fit LDA model to our dense matrix
lda = lda.fit(np.asarray(dense_matrix))

In [ ]:
# post-processing
# --------
# get feature names from our tf-idf vector
tf_feature_names = tf.get_feature_names_out()

# print top words 
print_top_words(lda,               # specify model
                tf_feature_names,  # specify feature names vector
                20)                # specify how many words we want to see
 

# now transform our data using the lda model and create a dataframe
topic_dist = lda.transform(tfidf_matrix)
topic_dist_df = pd.DataFrame(topic_dist).reset_index(drop = True)

# view the corresponding tf-idf dataframe with tf-idf values
topic_dist_df

We have 5 topics. 

Topic 1 revolves around  support (words: thank, help, need, care). 
Topic 2 revolves around action (words: act, today, vote, protect, pass). 
Topic 3 revolves around safety (words: violence, assault, end gun violence, gun, violence, hope)
Topic 4 revolves around democracy (words: impeachment, trump, president, election, democrats, voting, democracy, ballot, vote). 
Topic 5 means we need to clean and preprocess the data better to make better use of these functions. 


## Sentiment Analysis

* Next, let's analyze the sentiments contained within the tweets. You may use TextBlob or another library for these tasks. Do the following:

* Choose two legislators, one who you think will be more liberal and one who you think will be more conservative, and analyze their sentiment and/or subjectivity scores per tweet. For instance, you might do two scatterplots that plot each legislator's sentiment against their subjectivity, or two density plots for their sentiments. Do the scores match what you thought?
* Plot two more visualizations like the ones you chose in the first part, but do them to compare (1) Democrats v. Republicans and (2) Senators v. Representatives 

`TextBlob` has already been imported in the top cell.

In [ ]:
# create a feature colum of sentiment polarity and subjectivity
# ---------- 

# create new column feature of polarity
subset_df['polarity'] = subset_df['tokens_full_text'].map(lambda text: TextBlob(text).sentiment.polarity)

# create new column feature of subjectivity
subset_df['subjectivity'] = subset_df['tokens_full_text'].map(lambda text: TextBlob(text).sentiment.subjectivity)

# view
subset_df[['tokens_full_text', 'polarity']].head()

# histogram of polarity
sns.displot(subset_df, x="polarity") 
plt.show()

# plot
sns.displot(subset_df, x="subjectivity") 
plt.show()

In [ ]:
# loop to view the 5 most positive tweets
# ---------- 
for complaint in subset_df.nlargest(5, 'polarity')['text']:
    print(complaint + "\n")

In [ ]:
# loop to view the 5 most positive tweets
# ---------- 
for complaint in subset_df.nsmallest(5, 'polarity')['text']:
    print(complaint + "\n")

In [ ]:
# loop to view the 5 most subjective tweets
# ---------- 
for complaint in subset_df.nlargest(5, 'subjectivity')['text']:
    print(complaint + "\n")

In [ ]:
# loop to view the 5 most subjective tweets
# ---------- 
for complaint in subset_df.nsmallest(5, 'subjectivity')['text']:
    print(complaint + "\n")

Both of these measures look weird. Polarity is centered around 0 and subjectivity is centered around 0.5. However, we do find some difference when we do subgroup analysis. 

In [ ]:
## analyzing Steve Chabot (conservative) and Barbara Lee (democrat)

# Average polarity 
avg_polarity = subset_df['polarity'].mean()
print("Average Polarity for Full Sample:", avg_polarity)

# Average polarity by democrats and republicans
avg_polarity_by_party = subset_df.groupby('party')['polarity'].mean()
avg_polarity_republicans = avg_polarity_by_party.get('Republican', 0)  # 0 is the default if 'Republican' is not found
avg_polarity_democrats = avg_polarity_by_party.get('Democrat', 0)  # 0 is the default if 'Democratic' is not found
print("Average Polarity for Republicans:", avg_polarity_republicans)
print("Average Polarity for Democrats:", avg_polarity_democrats)

# Average polarity by representatives and senators
avg_polarity_by_position = subset_df.groupby('position')['polarity'].mean()
avg_polarity_representatives = avg_polarity_by_position.get('Rep', 0)  
avg_polarity_senators = avg_polarity_by_position.get('Sen', 0)  
print("Average Polarity for Representatives:", avg_polarity_representatives)
print("Average Polarity for Senators:", avg_polarity_senators)

# Group by 'name_wikipedia' and calculate the mean polarity
steve_chabot_avg_polarity = subset_df[subset_df['name_wikipedia'] == "Steve Chabot"]['polarity'].mean()
print("Average Polarity for Steve Chabot:", steve_chabot_avg_polarity)

# Group by 'name_wikipedia' and calculate the mean polarity
barbara_lee_avg_polarity = subset_df[subset_df['name_wikipedia'] == "Barbara Lee"]['polarity'].mean()
print("Average Polarity for Barbara Lee:", barbara_lee_avg_polarity)

###### 

# Average subjectivity  
avg_subjectivity = subset_df['subjectivity'].mean()
print("Average Subjectivity for Full Sample:", avg_subjectivity)

# Average polarity by democrats and republicans
avg_subjectivity_by_party = subset_df.groupby('party')['subjectivity'].mean()
avg_subjectivity_republicans = avg_polarity_by_party.get('Republican', 0)  # 0 is the default if 'Republican' is not found
avg_subjectivity_democrats = avg_polarity_by_party.get('Democrat', 0)  # 0 is the default if 'Democratic' is not found
print("Average Subjectivity for Republicans:", avg_subjectivity_republicans)
print("Average Subjectivity for Democrats:", avg_subjectivity_democrats)

# Average subjetivity by representatives and senators
avg_subjectivity_by_position = subset_df.groupby('position')['subjectivity'].mean()
avg_subjectivity_representatives = avg_polarity_by_position.get('Rep', 0)  
avg_subjectivity_senators = avg_polarity_by_position.get('Sen', 0)  
print("Average Subjectivity for Representatives:", avg_subjectivity_representatives)
print("Average Subjectivity for Senators:", avg_subjectivity_senators)

# Group by 'name_wikipedia' and calculate the mean subjectivity 
steve_chabot_avg_subjectivity = subset_df[subset_df['name_wikipedia'] == "Steve Chabot"]['subjectivity'].mean()
print("Average Subjectivity for Steve Chabot Waltz:", steve_chabot_avg_subjectivity)

# Group by 'name_wikipedia' and calculate the mean polarity
barbara_lee_avg_subjectivity = subset_df[subset_df['name_wikipedia'] == "Barbara Lee"]['subjectivity'].mean()
print("Average Subjectivity for Barbara Lee:", barbara_lee_avg_subjectivity)

We see a higher polarity score for Barbara Lee (liberal) relative to Steve Chabot (conservative), although it is pretty low for both of them so I don't know if I chose the best examples. Also, both of them are below the mean for the subjectivity index, and here again Barbara Lee (liberal) scores higher in subjectivity relative to Steve Chabot (conservative). I think I would have expected it to be the other way round. I also don't know how to interpret that 40% of the sample are being subjective, it seems a lot, but this is also tweets, right? It's the place for opinion, so perhaps it makes sense.  

In [ ]:
## scatter plot of polarity versus subjectivity for both legislators 

barbara_lee_data = subset_df[subset_df['name_wikipedia'] == 'Barbara Lee']
steve_chabot_data = subset_df[subset_df['name_wikipedia'] == 'Steve Chabot']

# Create scatter plot
plt.scatter(barbara_lee_data['subjectivity'], barbara_lee_data['polarity'], label='Barbara Lee')
plt.scatter(steve_chabot_data['subjectivity'], steve_chabot_data['polarity'], label='Steve Chabot')

# Add labels and title
plt.xlabel('Subjectivity')
plt.ylabel('Polarity')
plt.title('Scatter Plot of Polarity vs Subjectivity for Barbara Lee (liberal) vs Steve Chabot (conservative)')
plt.legend()

# Show plot
plt.show()

In [ ]:
## scatter plot of sentiment versus polarity for democrats vs republicans 

democrat = subset_df[subset_df['party'] == 'Democrat']
republican = subset_df[subset_df['party'] == 'Republican']

# Create scatter plot
plt.scatter(democrat['subjectivity'], democrat['polarity'], label='Democrat')
plt.scatter(republican['subjectivity'], republican['polarity'], label='Republican')

# Add labels and title
plt.xlabel('Subjectivity')
plt.ylabel('Polarity')
plt.title('Scatter Plot of Polarity vs Subjectivity for Democrats vs Republicans')
plt.legend()

# Show plot
plt.show()

In [ ]:
## scatter plot of sentiment versus polarity for representatives vs senators 

representative = subset_df[subset_df['position'] == 'Rep']
senator = subset_df[subset_df['position'] == 'Sen']

# Create scatter plot
plt.scatter(representative['subjectivity'], representative['polarity'], label='Representative')
plt.scatter(senator['subjectivity'], senator['polarity'], label='Senator')

# Add labels and title
plt.xlabel('Subjectivity')
plt.ylabel('Polarity')
plt.title('Scatter Plot of Polarity vs Subjectivity for Representatives vs Senators')
plt.legend()

# Show plot
plt.show()

The first graph is not very conclusive to me, but maybe it's because by plotting only two legislators we don't have enough observations to see a pattern? 
However, if we plot polarity against subjectivity for all democrats and all republicans, we see that there's pretty much perfect overlap, so that relationship is going in the same direction for both groups. 

* Plot two more visualizations like the ones you chose in the first part, but do them to compare (1) Democrats v. Republicans and (2) Senators v. Representatives 

In [ ]:
## Histograms that compare tweet length for republicans vs democrats and for senators vs representatives 

sns.displot(subset_df, 
            x="word_count", 
            hue="party", 
            col = "party") ## how do I tell it not to show independent without filtering the data before? 
plt.show()

sns.displot(subset_df, 
            x="word_count", 
            hue="position", 
           col = "position")
plt.show()

There are more tweets coming from Democrats relative to Republicans, but the distribution and average length are similar. 
There are more tweets coming from Representatives relative to Senators, but the distributions are also not that different. 

## Featurization

Before going to classification, explore different featurization techniques. Create three dataframes or arrays to represent your text features, specifically:

* Features engineered from your previous analysis. For example, word counts, sentiment scores, topic model etc.
* A term frequency-inverse document frequency matrix. 
* An embedding-based featurization (like a document averaged word2vec)

In the next section, you will experiment with each of these featurization techniques to see which one produces the best classifications.

### Engineered Text Features

In [ ]:
# Engineered Text Features: select engineered text features
engineered_features = subset_df[['tweet_length', 
                                 'word_count', 
                                 'polarity', 
                                 'subjectivity']].reset_index(drop = True)

engineered_features.head(10)

In [ ]:
# Topic Model + Engineered Features: combine results from our topic model + engineered features
engineered_features_with_topics = topic_dist_df.join(engineered_features.reset_index(drop = True))

engineered_features_with_topics.head(10)

### Bag-of-words or Tf-idf

In [ ]:
# create dataframe of tf-idf 
tfidf_df = pd.DataFrame(tfidf_matrix.todense(),                # specify matrix
                        columns = tf.get_feature_names_out())  # set feature names

tfidf_df.head(10)

### Word Embedding

In [ ]:
# Load Word2Vec model from Google; OPTIONAL depending on your computational resources (the file is ~1 GB)
# Also note that this file path assumes that the word vectors are underneath 'data'; you may wish to point to the CSS course repo and change the path
# or move the vector file to the project repo 

In [ ]:
from gensim.models import KeyedVectors

# Load the pre-trained Word2Vec model
model_path = 'data/GoogleNews-vectors-negative300.bin.gz'
word2vec_model = KeyedVectors.load_word2vec_format(model_path, binary=True)

In [ ]:
vector = word2vec_model['word']

model = gensim.models.Word2Vec(subset_df['tokens'],    # specify data - sentences
                               vector_size=100,        # set embedding size at 100
                               window=5,               # max distance between current and predicted word
                               min_count=5,            # ignores words with freq fewer than this threshold
                               sg=0,                   # specify Continuous Bag of Words Algorithim
                               alpha=0.025,            # learning rate
                               epochs = 5,             # iterations
                               seed = 12,              # set random seed (same as random_state in sklearn )
                               batch_words=10000,      # sample size 
                               workers = 1)  

In [ ]:
# create function to iterate over every token and document in our corpus
# ---------
def document_vector(word2vec_model, doc):
    # Filter out words that are not present in the vocabulary
    doc = [word for word in doc if word in word2vec_model.key_to_index]
    if doc:  # Check if doc is not empty
        return np.mean(word2vec_model.__getitem__(doc), axis=0)
    else:
        # Return None or any default value if doc is empty
        return np.zeros(word2vec_model.vector_size)  # Return zero vector if doc is empty

In [ ]:
# create an array for the size of the corpus
# ----------
# create empty list
empty_list_embeddings_means = []

# loop over each each token
for doc in subset_df['tokens']: # append the vector for each document
    empty_list_embeddings_means.append(document_vector(word2vec_model, doc))

# convert the list to array
doc_average_embeddings = np.array(empty_list_embeddings_means) 

# make df of averages
word2vec_df = pd.DataFrame(doc_average_embeddings)

In [ ]:
# return a list of word for which we have calculations
# ----------
words = list(model.wv.index_to_key)
print(words[0:100])  # print the first 100 words

In [ ]:
## just for fun 

# similarity between Democrat and Republican
# ----------
similarity_democrat_republican = model.wv.similarity('Democrat', 'Republican')
print("Similarity between democrat and republican:", similarity_democrat_republican)

# similarty between act and speak
# ----------
similarity_act_speak = model.wv.similarity('act', 'speak')
print("Similarity between act and speak:", similarity_act_speak)

# which doesn't belong? 1
# ----------
which_does_not_belong = model.wv.doesnt_match(['Democrat', 'Republican', 'Americans']) ## so interesting that it's picking Democrat here as the word that does NOT belong 
print("Which does not belong between Democrat, Republcian and Americans?:", which_does_not_belong)

# vector math
# ----------
most_similar = model.wv.most_similar(positive=['Democrat', 'act', 'today'], 
                      negative=['impeachment', 'law'])
print("Which is most similar?:", most_similar)

## Classification

Either use cross-validation or partition your data with training/validation/test sets for this section. Do the following:

* Choose a supervised learning algorithm such as logistic regression, random forest etc. 
* Train six models. For each of the three dataframes you created in the featurization part, train one model to predict whether the author of the tweet is a Democrat or Republican, and a second model to predict whether the author is a Senator or Representative.
* Report the accuracy and other relevant metrics for each of these six models.
* Choose the featurization technique associated with your best model. Combine those text features with non-text features. Train two more models: (1) A supervised learning algorithm that uses just the non-text features and (2) a supervised learning algorithm that combines text and non-text features. Report accuracy and other relevant metrics. 

If time permits, you are encouraged to use hyperparameter tuning or AutoML techniques like TPOT, but are not explicitly required to do so.

### Train Six Models with Just Text

In [ ]:
# six models ([engineered features, frequency-based, embedding] * [democrat/republican, senator/representative])

In [ ]:
# model = LogisticRegression(max_iter=500)
# df = dataframes[0]
# model.fit(df.to_numpy(), labels[0][0].argmax(axis=1))

In [ ]:
# Initialize a list of dataframes from the objects created above
dataframes = [engineered_features, tfidf_df]
dataframe_names = ['engineered_features', 'tfidf_df']

# Create label binarizer for party (Democrat/Republican)
party_lb = LabelBinarizer()
party_labels = party_lb.fit_transform(subset_df["party"])  # Binarize party labels

# Create label binarizer for party (Democrat/Republican)
position_lb = LabelBinarizer()
position_labels = position_lb.fit_transform(subset_df["position"])  # Binarize position labels

labels = [(party_labels, 'Party Affiliation'), (position_labels, 'Position')]

In [ ]:
## I cannot get the function to work so I'm doing one by one. Sorry about that!     

# engineered_features - party 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(engineered_features, 
                                                    party_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
# engineered_features - position 

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(engineered_features, 
                                                
                                                    position_labels, 
                                                
                                                    test_size=0.2, 
                                                
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## tfidf_df - party
    
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, 
                                                    party_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## tfidf_df - position

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(tfidf_df, 
                                                    position_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## word_embeddings - party

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(doc_average_embeddings, 
                                                    party_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## word_embeddings - position

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(doc_average_embeddings, 
                                                    position_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

### Two Combined Models

In [ ]:
# two models ([best text features + non-text features] * [democrat/republican, senator/representative])

## tf-idf worked better then engineered features, so I'm comibing this df with text features and non text features. 

# Non-text features
# --------
# select non-text features
non_text_features = subset_df[['state', 
                               'position', 
                               'gender',
                               'birthday', 
                               'joined_congress_date', 
                               'clinton_2016_state_share',
                               'obama_2012_state_share', 
                               'romney_2012_state_share']]

# get dummies of non-text features
non_text_features_dummies = pd.get_dummies(non_text_features).reset_index(drop = True)

# combine non-text features with tf-idf  
non_text_plus_tfidf = non_text_features_dummies.reset_index(drop = True).join(tfidf_df)

In [ ]:
## non-text features - party

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(non_text_features_dummies, 
                                                    party_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## tfidf_df non-text features - party

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(non_text_plus_tfidf, 
                                                    party_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## non-text features - position

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(non_text_features_dummies, 
                                                    position_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

In [ ]:
## tfidf_df non-text features - position

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(non_text_plus_tfidf, 
                                                    position_labels, 
                                                    test_size=0.2, 
                                                    random_state=42)
# Initialize the model
model = LogisticRegression(max_iter=500)

# Train the model
model.fit(X_train,    
            y_train.ravel())

# Predict labels for the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Calculate confusion matrix
confusion_mat = confusion_matrix(y_test, y_pred)

# Calculate classification report
class_report = classification_report(y_test, y_pred)

# Print the results
print(f'Accuracy: {accuracy}')
print(f'Confusion Matrix:\n{confusion_mat}')
print(f'Classification Report:\n{class_report}\n')

## Discussion Questions

1. Why do standard preprocessing techniques need to be further customized to a particular corpus?

Standard preprocessing techniques provide a good starting point for text analysis. However, it's necessary to adapt these techniques to the specific needs and context of your dataset to be able to perform a more accurate analysis. Some examples might be the following: 

** Domain-specific terminology or jargon, such as abbreviations or acronyms.

** Irregularities, such as misspellings, grammatical errors, or slang, or the use of informal language. 

** Sentiment analysis may require handling negation, sarcasm, formality or emoticons, among others.

2. Did you find evidence for the idea that Democrats and Republicans have different sentiments in their tweets? What about Senators and Representatives?

Yes, we can see that republicans have, on average, a higher polarity score (0.14 republicans vs 0.10 democrats) and higher subjectivity score (again, 0.14 republicans vs. 0.10 democrats). In the particular example given the legislators that I randomly picked, I found it was the other way round, meaning that the democratic legislator had both a higher polarity and subjectivity score, but perhaps I did not pick good examples that extrapolate over the overall sample. We do not see, however, the same trend for representatives and senators. Both representatives and senators show a polarity and subjectivity score of 0.11. 

3. Why is validating your exploratory and unsupervised learning approaches with a supervised learning algorithm valuable?

Validating exploratory and unsupervised learning approaches with a supervised learning algorithm can be valuable for the following reasons: 

** Because supervised learning algorithms are trained on labeled data, they allow for a more objective assessment of the effectiveness and accuracy of the unsupervised methods.

** Exploratory and unsupervised learning methods help reveal hidden patterns in data. When we confirm these findings using supervised learning algorithms, we might get better insights or it might give us a better understanding of the data's underlying structure.

** Validating unsupervised techniques with supervised models helps achieve generalizability of the insights and patterns to be applied to unseen data. 

** Validating unsupervised techniques with supervised models can help in explaining relationships between features and target variables.

4. Did text only, non-text only, or text and non-text features together perform the best? What is the intuition behind combining text and non-text features in a supervised learning algorithm?

Using text only has a lower accuracy score (73% for party and 77% for position), but either using non-text features or using text and non-text features combined has an accuracy score of 98% or higher. Intuition: Combining text and non-text features in a supervised learning algorithm allows us to take advantage of the strengths of both types of features. Text features capture the linguistic content and semantics of the data, while non-text features provide additional context or domain knowledge that we cannot get from text alone. By combining both, we can improve predictive performance. What is not so clear to me here (maybe I made a coding mistake) is why non-text features alone was better than text alone. 